# Day 5: Cafeteria ---
```
As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.

You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.

"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.

The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are fresh and which are spoiled. When you ask how it works, they give you a copy of their database (your puzzle input).

The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:

3-5
10-14
16-20
12-18

1
5
8
11
17
32
The fresh ID ranges are inclusive: the range 3-5 means that ingredient IDs 3, 4, and 5 are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

The Elves are trying to determine which of the available ingredient IDs are fresh. In this example, this is done as follows:

Ingredient ID 1 is spoiled because it does not fall into any range.
Ingredient ID 5 is fresh because it falls into range 3-5.
Ingredient ID 8 is spoiled.
Ingredient ID 11 is fresh because it falls into range 10-14.
Ingredient ID 17 is fresh because it falls into range 16-20 as well as range 12-18.
Ingredient ID 32 is spoiled.
So, in this example, 3 of the available ingredient IDs are fresh.

Process the database file from the new inventory management system. How many of the available ingredient IDs are fresh?
```

In [6]:
import sys
sys.path.append("..")

from common.utils import Day, read_lines
test_path = "test.txt"
data_path = "data.txt"

In [7]:
class Day5(Day):
    def __init__(self, path):
        super().__init__(path)
        self.inventory = read_lines(path)

        # Separate ID ranges from ingredients:
        delimiter_pos = self.inventory.index("")
        self.id_ranges_str = self.inventory[:delimiter_pos]
        self.id_ranges = self._get_ranges()

        self.ingredients = self.inventory[(delimiter_pos + 1):]
        self.ingredients = [int(n) for n in self.ingredients]
        pass

    def _get_ranges(self):
        """
        Turn strings into range objects
        """

        id_ranges = []
        for id_str in self.id_ranges_str:
            start, stop = id_str.split("-")

            # NOTE, Range objects generate start to stop-1 in python
            id_range = range(int(start), int(stop ) + 1, 1)
            id_ranges.append(id_range)
   
        return id_ranges
    
    def solve_part_1(self):
        """
         Python has in-build logic for this operation:
            r = range(1, 400)
            400 in r
                returns False
            250 in r
                returns True
        """
        result = 0
        for i in self.ingredients:
            for r in self.id_ranges:
                if i in r:
                    result +=1
                    break

        return result

day5 = Day5(test_path)
day5.solve_part_1()

3

In [8]:
day5 = Day5(data_path)
day5.solve_part_1()

520

# Part Two ---
```
The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory, the Elves would like to know all of the IDs that the fresh ingredient ID ranges consider to be fresh. An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:

3-5
10-14
16-20
12-18
The ingredient IDs that these ranges consider to be fresh are 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, and 20. So, in this example, the fresh ingredient ID ranges consider a total of 14 ingredient IDs to be fresh.

Process the database file again. How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?`
```

In [83]:
%%time
"""
Clever way:
    find a way to avoid duplicates and just cound the lenght of the non-overlapping intervals

Brute force: DOES NOT WORK
    create a set() and add all elements from each range
"""

class Day5b(Day5):
    def __init__(self, path):
        super().__init__(path)
        pass
    
    def merge_intervals(self):
        """
        I found a clever algorithm that works with tupples:
        https://stackoverflow.com/questions/15273693/union-of-multiple-ranges
        a = [(7, 10), (11, 13), (11, 15), (14, 20), (23, 39)]
        b = []
        for begin,end in sorted(a):
            if b and b[-1][1] >= begin - 1:
                b[-1][1] = max(b[-1][1], end)
            else:
                b.append([begin, end])

        b -> [(7, 20), (23, 39)]
        """

        intervals = []

        # Convert "10-39" strings → (10, 39) tuples
        for id_str in self.id_ranges_str:
            start, stop = map(int, id_str.split("-"))
            intervals.append((start, stop))

        intervals.sort()

        merged = []
        for start, end in intervals:
            if merged and merged[-1][1] >= start - 1:  # overlap or touching
                merged[-1] = (merged[-1][0], max(merged[-1][1], end))
            else:
                merged.append((start, end))

        return merged


    def solve_part_2(self):
        """
        Create a list of non-overlapping instervals as tupples
        The number of all possible ids that are fresh is
        the number of all elements in all ranges (both inclusive)
        length of an interval (inclusive) is:
            l = stop - start + 1 
        """
        merged = self.merge_intervals()
        total = sum(stop - start + 1 for start, stop in merged)
        return total
        
day5 = Day5b(test_path)
day5.solve_part_2()

CPU times: total: 0 ns
Wall time: 0 ns


14

In [95]:
%%timeit
day5 = Day5b(data_path)
day5.solve_part_2()

690 μs ± 25 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


####  325122487952659 is too low for some reason

#### 347338785050519 is too high

Those damn Range values and "off-by-one" errors

---- 

I ended up working with tupples instead, so not off by one errors and such

In [ ]:
"""
NEXT IDEA:
the length of an interval is easy to calculate:
 l = stop - start 

 if all intervals where non-overlapping then we would be okay.

-> generate a list of non-overlapping ranges, then add their length.

Possiblitiies:
    given A and B 2 range objects:
        A and B are non-overlapping (A U B = A + B)
            - we keep both A and B
        A is inside B (A + B = B)
            Discard A, keep B
        A and B have a common range of values (|A + B| < |A| + |B|)
            Replace both A and B with [min(startA, startB), max(stopA, stopB)]

        do this untill all intervals are non-overlapping
"""